In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

C:\Users\iltya\AppData\Roaming\Python\Python39\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = pd.read_excel("data/covid_UPD.xlsx")

In [3]:
df.shape

(76219, 22)

проверим на пропущенные значения

In [4]:
df.head()

,week,region,oced,unemployed,cv_gender,cv_birthday_2000,cv_birthday_1990,cv_birthday_1980,cv_birthday_1970,cv_birthday_1960,...,women,infections,deaths,healed,tests,Rt,level,yandex_continuous,yandex_level,pass
0,2020-04-06,Адыгея,"Строительство, ремонт, стройматериалы, недвижи...",19,18,0,3,12,19,19,...,245928.0,42.0,0.0,44.0,NaN,0.873684,NaN,3.314286,2.0,0.0
1,2020-04-06,Адыгея,Машиностроение,2,1,0,0,1,1,2,...,245928.0,42.0,0.0,44.0,NaN,0.873684,NaN,3.314286,2.0,0.0
2,2020-04-06,Адыгея,Юриспруденция,13,6,0,2,11,13,13,...,245928.0,42.0,0.0,44.0,NaN,0.873684,NaN,3.314286,2.0,0.0
3,2020-04-06,Адыгея,Пищевая промышленность,9,2,1,2,6,9,9,...,245928.0,42.0,0.0,44.0,NaN,0.873684,NaN,3.314286,2.0,0.0
4,2020-04-06,Адыгея,"Государственная служба, некоммерческие организ...",17,8,0,3,14,17,17,...,245928.0,42.0,0.0,44.0,NaN,0.873684,NaN,3.314286,2.0,0.0


### из-за низкой репрезентативности удалим регионы с суммарным кол-вом поданных заявок меньше 1000

In [5]:
count_sum_unemployed_by_regions = df.groupby("region").agg(sum_uneployed=("unemployed", "sum")).sort_values(by="sum_uneployed")
count_sum_unemployed_by_regions

,sum_uneployed
region,
Байконур,63
Чукотский АО,144
Ненецкий АО,820
Магаданская,1369
Сахалинская,4892
...,...
Ростовская,149789
Московская,191943
Краснодарский край,224576


In [16]:
df_unemployed_by_regions = df.groupby("region").agg(sum_uneployed=("unemployed", "sum"),
                                                           population=("population", "mean")).dropna()

In [17]:
df_unemployed_by_regions["procent_unamployed"] = df_unemployed_by_regions.sum_uneployed / df_unemployed_by_regions.population

In [19]:
df_unemployed_by_regions

,sum_uneployed,population,procent_unamployed
region,,,
Адыгея,17898,463088.0,0.038649
Алтай,15186,220181.0,0.068971
Алтайский край,47654,2317153.0,0.020566
Амурская,25986,790044.0,0.032892
Архангельская,22140,1092424.0,0.020267
...,...,...,...
Чечня,226264,1478726.0,0.153013
Чувашия,42008,1217818.0,0.034494
Чукотский АО,144,50288.0,0.002864


In [5]:
# regions_to_del = count_sum_unemployed_by_regions[count_sum_unemployed_by_regions.sum_uneployed < 1000].index
# df = df.drop(df[df.region.isin(regions_to_del)].index, axis=0)
# df.shape

In [21]:
regions_to_del = ["Байконур", "Чечня", "Чукотский АО"]
df = df.drop(df[df.region.isin(regions_to_del)].index, axis=0)

### также посмотрим на колво заражений, смертей и выздоравлений в разных регионах

In [23]:
covid_df = df.groupby(["region", "week"]).agg(sum_infections = ("infections", "mean"), population=("population", "mean"), sum_healed=("healed", "mean"), sum_deathed=("deaths", "mean"))

In [24]:
covid_df["procent_healed"] = covid_df.sum_healed / covid_df.sum_infections
covid_df["procent_infected"] = covid_df.sum_infections / covid_df.population
covid_df["procent_death"] = covid_df.sum_deathed / covid_df.population

In [25]:
covid_df

sum_infections  population  sum_healed  sum_deathed  \
region      week                                                              
Адыгея      2020-04-06            42.0    463088.0        44.0          0.0   
            2020-04-13           278.0    463088.0       233.0          2.0   
            2020-04-20            80.0    463088.0        11.0          2.0   
            2020-04-27            42.0    463088.0        16.0          1.0   
            2020-05-04            21.0    463088.0        16.0          0.0   
...                                ...         ...         ...          ...   
Ярославская 2020-09-28           345.0   1253389.0       311.0          1.0   
            2020-10-05           167.0   1253389.0        81.0          0.0   
            2020-10-12           611.0   1253389.0       809.0          0.0   
            2020-10-19           677.0   1253389.0       440.0          2.0   
            2020-10-26           770.0   1253389.0       566.0          6.0   

                        procent_healed  procent_infected  procent_death  
region      week                                                         
Адыгея      2020-04-06        1.047619          0.000091   0.000000e+00  
            2020-04-13        0.838129          0.000600   4.318834e-06  
            2020-04-20        0.137500          0.000173   4.318834e-06  
            2020-04-27        0.380952          0.000091   2.159417e-06  
            2020-05-04        0.761905          0.000045   0.000000e+00  
...                                ...               ...            ...  
Ярославская 2020-09-28        0.901449          0.000275   7.978369e-07  
            2020-10-05        0.485030          0.000133   0.000000e+00  
            2020-10-12        1.324059          0.000487   0.000000e+00  
            2020-10-19        0.649926          0.000540   1.595674e-06  
            2020-10-26        0.735065          0.000614   4.787021e-06  

[2490 rows x 7 columns]

В Москве суммарно число выявленных случаев заболевших больше чем численность населения на начало 2020 года - это может быть связано с повторным заражением инфекцией

In [30]:
df.shape

(75094, 22)

In [32]:
df.to_csv("data/dataset_1.csv", index=None)